In [0]:
# Ingestão de Cards - Magic: The Gathering (Versão Corrigida)
# Objetivo: Ingerir dados de cards da API do Magic: The Gathering para staging em Parquet no S3
# Características: Dados brutos, formato Parquet, filtro temporal, particionamento, incremental, paginado

# =============================================================================
# BIBLIOTECAS UTILIZADAS
# =============================================================================
import requests
import json
import logging
from datetime import datetime, timedelta
from pyspark.sql.functions import current_timestamp, lit, col, year, month, when
from pyspark.sql.types import *
import time

# =============================================================================
# CONFIGURAÇÃO DE SEGREDOS
# =============================================================================

def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Segredo '{secret_name}' não configurado")

# =============================================================================
# CONFIGURAÇÕES GLOBAIS
# =============================================================================

# Configuração de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configurações da API
API_BASE_URL = get_secret("api_base_url", "https://api.magicthegathering.io/v1")
BATCH_SIZE = int(get_secret("batch_size", "100"))
MAX_RETRIES = int(get_secret("max_retries", "3"))

# Configurações do S3
S3_BUCKET = get_secret("s3_bucket", "lakenokiseki")
S3_PREFIX = get_secret("s3_prefix", "magic_the_gathering/stage")
S3_BASE_PATH = f"s3://{S3_BUCKET}/{S3_PREFIX}"

# Configurações de período
YEARS_BACK = int(get_secret("years_back", "5"))
current_year = datetime.now().year
cutoff_year = current_year - YEARS_BACK
CUTOFF_DATE = datetime(cutoff_year, 1, 1)
CUTOFF_DATE_STR = CUTOFF_DATE.strftime("%Y-%m-%d")

# Log das configurações
print("=" * 60)
print("CONFIGURAÇÕES PARA INGESTÃO DE CARDS")
print("=" * 60)
print("API_BASE_URL: [CONFIGURADO]")
print("S3_BASE_PATH: [CONFIGURADO]")
print(f"YEARS_BACK: {YEARS_BACK}")
print(f"CUTOFF_DATE_STR: {CUTOFF_DATE_STR}")
print("=" * 60)



In [0]:
# =============================================================================
# FUNÇÕES UTILITÁRIAS
# =============================================================================

def setup_s3_storage():
    try:
        # Verificar se o diretório existe (criar se necessário)
        try:
            dbutils.fs.ls(S3_BASE_PATH)
            print("Diretório do S3 já existe")
        except:
            dbutils.fs.mkdirs(S3_BASE_PATH)
            print("Diretório do S3 criado com sucesso")
        
        return True
        
    except Exception as e:
        print(f"Erro ao configurar S3 storage: {e}")
        return False

def make_api_request(endpoint, params=None, retries=MAX_RETRIES):
    url = f"{API_BASE_URL}/{endpoint}"
    print("Fazendo requisição para API...")
    
    for attempt in range(retries):
        try:
            response = requests.get(url, params=params, timeout=30)
            
            if response.status_code == 200:
                data = response.json()
                return data
            elif response.status_code == 429:  # Rate limit
                wait_time = min((attempt + 1) * 5, 60)
                print(f"Rate limit atingido. Aguardando {wait_time}s...")
                time.sleep(wait_time)
            elif response.status_code == 503:  # Service unavailable
                wait_time = min((attempt + 1) * 10, 120)
                print(f"Serviço indisponível. Aguardando {wait_time}s...")
                time.sleep(wait_time)
            else:
                print(f"Erro {response.status_code} na API: {response.text[:200]}")
                if attempt < retries - 1:
                    time.sleep(5)
                
        except requests.exceptions.Timeout:
            print("Timeout na requisição")
            if attempt < retries - 1:
                time.sleep(10)
        except requests.exceptions.RequestException as e:
            if attempt == retries - 1:
                print(f"Erro na requisição para endpoint: {e}")
                return None
            print("Tentativa falhou, tentando novamente...")
            time.sleep(1)
        except json.JSONDecodeError as e:
            print(f"Erro ao decodificar JSON: {e}")
            if attempt == retries - 1:
                return None
            time.sleep(1)
    
    return None

def clean_cards_data(data):
    cleaned_data = []
    for item in data:
        if isinstance(item, dict):
            cleaned_item = {}
            
            # Mapear campos conhecidos com tipos seguros
            field_mappings = {
                'name': str,
                'manaCost': str,
                'cmc': float,
                'type': str,
                'rarity': str,
                'set': str,
                'setName': str,
                'text': str,
                'artist': str,
                'number': str,
                'power': str,
                'toughness': str,
                'layout': str,
                'multiverseid': int,
                'imageUrl': str,
                'originalText': str,
                'originalType': str,
                'id': str
            }
            
            # Processar campos conhecidos
            for field, field_type in field_mappings.items():
                if field in item:
                    try:
                        if item[field] is not None:
                            cleaned_item[field] = field_type(item[field])
                        else:
                            cleaned_item[field] = None
                    except (ValueError, TypeError):
                        # Se não conseguir converter, usar string
                        cleaned_item[field] = str(item[field]) if item[field] is not None else None
                else:
                    cleaned_item[field] = None
            
            # Tratar campos de lista - converter para string JSON
            list_fields = ['colors', 'colorIdentity', 'types', 'subtypes', 'variations', 'foreignNames', 'printings', 'legalities']
            for field in list_fields:
                if field in item and item[field] is not None:
                    if isinstance(item[field], list):
                        cleaned_item[field] = json.dumps(item[field])
                    else:
                        cleaned_item[field] = str(item[field])
                else:
                    cleaned_item[field] = None
            
            cleaned_data.append(cleaned_item)
    
    return cleaned_data

def save_to_parquet(data, table_name):
    if not data:
        print(f"Nenhum dado para salvar na tabela {table_name}")
        return None
    try:
        if table_name == 'cards':
            # Schema explícito para cards
            schema_fields = [
                StructField("name", StringType(), True),
                StructField("manaCost", StringType(), True),
                StructField("cmc", FloatType(), True),
                StructField("colors", StringType(), True),
                StructField("colorIdentity", StringType(), True),
                StructField("type", StringType(), True),
                StructField("types", StringType(), True),
                StructField("subtypes", StringType(), True),
                StructField("rarity", StringType(), True),
                StructField("set", StringType(), True),
                StructField("setName", StringType(), True),
                StructField("text", StringType(), True),
                StructField("artist", StringType(), True),
                StructField("number", StringType(), True),
                StructField("power", StringType(), True),
                StructField("toughness", StringType(), True),
                StructField("layout", StringType(), True),
                StructField("multiverseid", IntegerType(), True),
                StructField("imageUrl", StringType(), True),
                StructField("variations", StringType(), True),
                StructField("foreignNames", StringType(), True),
                StructField("printings", StringType(), True),
                StructField("originalText", StringType(), True),
                StructField("originalType", StringType(), True),
                StructField("legalities", StringType(), True),
                StructField("id", StringType(), True)
            ]
            
            schema = StructType(schema_fields)
            df = spark.createDataFrame(data, schema)
        else:
            df = spark.createDataFrame(data)
        
        df = df.withColumn("ingestion_timestamp", current_timestamp()) \
               .withColumn("source", lit("mtg_api")) \
               .withColumn("endpoint", lit(table_name))
        
        # Particionamento por data atual (cards não têm data de lançamento individual)
        df = df.withColumn("partition_year", lit(datetime.now().year)) \
               .withColumn("partition_month", lit(datetime.now().month))
        
        partition_combinations = df.select("partition_year", "partition_month").distinct().collect()
        for partition_row in partition_combinations:
            partition_year = partition_row["partition_year"]
            partition_month = partition_row["partition_month"]
            partition_df = df.filter((col("partition_year") == partition_year) & 
                                   (col("partition_month") == partition_month))
            file_name = f"{partition_year}_{partition_month:02d}_{table_name}.parquet"
            file_path = f"{S3_BASE_PATH}/{file_name}"
            try:
                existing_files = dbutils.fs.ls(file_path)
                if len(existing_files) > 0:
                    print(f"Arquivo {file_name} já existe - pulando")
                    continue
            except:
                pass
            partition_df.drop("partition_year", "partition_month").write.mode("overwrite").format("parquet").save(file_path)
            print(f"Arquivo {file_name} criado com sucesso")
        print(f"Registros salvos como Parquet para {table_name}")
        return df
    except Exception as e:
        print(f"Erro ao salvar dados em {table_name}: {e}")
        return None

def ingest_paginated_data(endpoint, table_name, data_key=None, max_pages=100):
    print(f"Iniciando ingestão paginada: {table_name}")
    
    if data_key is None:
        data_key = table_name
    
    all_data = []
    page = 1
    
    while page <= max_pages:
        print("Processando página...")
        
        params = {"page": page, "pageSize": BATCH_SIZE}
        data = make_api_request(endpoint, params)
        
        if data and data_key in data:
            page_data = data[data_key]
            print(f"Processando página {page}...")
            
            if not page_data:
                print("Página vazia - fim da paginação")
                break
            
            # Limpar dados se for cards
            if table_name == 'cards':
                print("Limpando dados de cards...")
                page_data = clean_cards_data(page_data)
                print(f"Limpando dados de cards...")
            
            all_data.extend(page_data)
            page += 1
            
            # Aguardar um pouco entre as requisições
            time.sleep(0.5)
        else:
            print("Falha ao obter dados da página")
            break
    

    
    # Verificar se há dados antes de tentar salvar
    if not all_data:
        print(f"Nenhum dado válido para {table_name}")
        return None
    
    try:
        df = save_to_parquet(all_data, table_name)
        
        if df:
            count = df.count()
            print(f"{table_name}: {count} registros processados")
            display(df.limit(5))
        return df
    except Exception as e:
        print(f"Erro ao processar dados de {table_name}: {e}")
        return None

# Verificar Spark
try:
    spark
    print("Spark disponível")
except NameError:
    print("Spark não está disponível - tentando obter do contexto")
    try:
        from pyspark.sql import SparkSession
        spark = SparkSession.builder.getOrCreate()
        print("Spark criado com sucesso")
    except Exception as e:
        print(f"Erro ao criar Spark: {e}")
        raise Exception("Spark não está disponível")

# Configurar S3 Storage
setup_success = setup_s3_storage()
if not setup_success:
    raise Exception("Falha ao configurar S3 storage")

print("Setup concluído com sucesso")



In [0]:
# Iniciar ingestão de cards
print("Iniciando ingestão de cards...")

cards_df = ingest_paginated_data(
    endpoint="cards",
    table_name="cards",
    max_pages=100  # Limite de 100 páginas para demonstração
)

# Gerar relatório
print("=" * 50)
print("RELATÓRIO DE INGESTÃO DE CARDS")
print("=" * 50)

if cards_df:
    print("Arquivos salvos com sucesso")
else:
    print("Falha na ingestão de cards")

print("=" * 50)

# Ingestão de Cards Concluída!
# Resumo: Tabela cards, registros processados, arquivos salvos em S3
# Como acessar: df = spark.read.parquet("[S3_PATH]/cards_*.parquet") 